In [3]:
import os

In [4]:

os.chdir("../")

In [10]:
pwd

'c:\\Users\\youss\\OneDrive\\Bureau\\cl'

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    trained_model_path: Path
    MLflow_URI: Path 
    training_data: Path
    all_params: dict
    params_batch_size: int
    params_image_size: list



In [26]:
from classification.constants import *
from classification.utils.common import read_yaml,create_directories,  save_json

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_Evaluation_config(self) -> EvaluationConfig:
        Evaluation= self.config.Evaluation
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Data")
        create_directories([
            Path(Evaluation.root_dir)
        ])
        


        Evaluation_config = EvaluationConfig(
            root_dir=Path(Evaluation.root_dir),
            trained_model_path=Path(Evaluation.trained_model_path),
            MLflow_URI=Evaluation.MLflow_URI,
            training_data=Path(training_data),

            all_params=params,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )
        return Evaluation_config

In [28]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yusufM03/Classification_Mlflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="yusufM03"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3735ecba8f2f0dccee05c421728f3ed8abd06c96"





In [53]:
import os
import tensorflow as tf
from classification import logger
import mlflow
import mlflow.keras
import numpy as np
from classification.components.Training_model import training_Model 
#Component 
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder

class Evaluate_Model:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def Processing_Data(self):

      
  
        val_Data=os.path.join(self.config.training_data,'test')
        X_val=[]
        y_val=[]

        for classe in os.listdir(val_Data):
            
          for img in os.listdir(os.path.join(val_Data,classe)) :
            img_path=os.path.join(val_Data,classe,img)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(self.config.params_image_size[0], self.config.params_image_size[0]))
            x = tf.keras.preprocessing.image.img_to_array(img)

            x = tf.keras.applications.vgg16.preprocess_input(x)
            X_val.append(x)
            y_val.append(classe)

        y_val_array = np.array(y_val)
        y_val_reshaped = y_val_array.reshape(-1, 1)
      

        # Initialize the one-hot encoder
        onehot_encoder = OneHotEncoder(sparse=False)

        y_val_onehot =onehot_encoder.fit_transform(y_val_reshaped)

        x_val=np.array(X_val)

        self.val=x_val
        self.y_val=y_val_onehot
      
      
        

    

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.trained_model_path)

        self.score = self.model.evaluate(self.val,self.y_val)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


  
    def log_into_mlflow(self):
      mlflow.set_registry_uri(self.config.MLflow_URI)
      tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
      
      with mlflow.start_run():
          mlflow.log_params(self.config.all_params)
          mlflow.log_metrics(
              {"loss": self.score[0], "accuracy": self.score[1]}
          )
          # Model registry does not work with file store
          if tracking_url_type_store != "file":

              # Register the model
              # There are other ways to use the Model Registry, which depends on the use case,
              # please refer to the doc for more information:
              # https://mlflow.org/docs/latest/model-registry.html#api-workflow
              mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
          else:
              mlflow.keras.log_model(self.model, "model")
    

In [54]:
try:
    config = ConfigurationManager()
    Evaluate_model_config = config.get_Evaluation_config()
    Training_model = Evaluate_Model(config=Evaluate_model_config )
    Training_model.load_model("artifacts/training/model.h5")
    Training_model.Processing_Data()
    Training_model.evaluation()
    Training_model.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-24 06:09:32,736: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-24 06:09:32,741: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-24 06:09:32,743: INFO: common: created directory at: artifacts]
[2024-03-24 06:09:32,744: INFO: common: created directory at: artifacts\training]


c:\Users\youss\anaconda3\envs\classification_cancer\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


10/10 [==============================] - 22s 2s/step - loss: 1054.6379 - accuracy: 0.5048
[2024-03-24 06:09:56,975: INFO: common: json file saved at: scores.json]


2024/03/24 06:09:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-03-24 06:10:00,044: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-03-24 06:10:00,845: INFO: builder_impl: Assets written to: C:\Users\youss\AppData\Local\Temp\tmpmzdtwocv\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/03/24 06:11:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
